In [1]:
import cv2
import imutils
import numpy as np
import os
from sklearn.svm import LinearSVC
from sklearn.externals import joblib
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [62]:
train_path = "../KDEF/"
training_names = os.listdir(train_path)

In [63]:
def imlist(path):
    """
    The function imlist returns all the names of the files in 
    the directory path supplied as argument to the function.
    """
    return [path+'/'+f for f in os.listdir(path)]

In [64]:
# Get all the path to the images and save them in a list
# image_paths and the corresponding label in image_paths
image_paths = []
image_classes = []
class_id = 0
for training_name in training_names:
    dir = os.path.join(train_path, training_name)
    class_path = imlist(dir)
    image_paths += class_path
    image_classes += [class_id] * len(class_path)
    class_id += 1

In [65]:
# 创建SIFT特征提取器
sift = cv2.xfeatures2d.SIFT_create()

# 特征提取与描述子生成
des_list = []
order = 0

for image_path in image_paths:
    if order % 5 >= 2:
        im = cv2.imread(image_path)
        # im = cv2.resize(im, (300, 300))
        kpts = sift.detect(im) #寻找关键点
        kpts, des = sift.compute(im, kpts) #计算关键点描述子
        des_list.append((image_path, des))
    order = order + 1
    # print("image file path : ", image_path)

In [92]:
type(des_list[0][1])

numpy.ndarray

In [93]:
# 描述子向量
# descriptors = des_list[0][1]
# for image_path, descriptor in des_list[1:]:
#     print(type(descriptor))
#     descriptors = np.vstack((descriptors, descriptor)) #堆叠

descriptors = des_list[0][1]
for i in range(1, len(des_list)):
    if des_list[i][1] is not None:
        descriptors = np.vstack((descriptors, des_list[i][1])) #堆叠
    else:
        descriptors = np.vstack((descriptors, des_list[i%3][1]))

In [115]:
# 聚类 K-Means
k = 300 #聚类中心的个数
voc, variance = kmeans(descriptors, k, 1)

In [145]:
# 生成特征直方图
im_features = np.zeros((len(des_list), k), "float32")
for i in range(len(des_list)):
    if des_list[i][1] is not None:
        words, distance = vq(des_list[i][1], voc)
    else:
        words, distance = vq(des_list[i%3][1], voc)
    for w in words:
        im_features[i][w] += 1

In [146]:
nbr_occurences = np.sum((im_features > 0) * 1, axis=0) #每个词出现的次数统计
idf = np.array(np.log((1.0 * len(image_paths) + 1) / (1.0 * nbr_occurences + 1)), 'float32') #词频计算

In [147]:
im_features = im_features.astype(np.int32)

In [148]:
np.savetxt("sift.csv", im_features.astype(np.int32), delimiter=',')

In [153]:
np.shape(image_paths)

(4900,)